In [1]:
import tkinter as tk
import os
import pandas as pd
from tkinter import filedialog
import numpy as np

# Data collection

In [2]:
# Créer une instance de la fenêtre principale
root = tk.Tk()
root.withdraw()  # Cacher la fenêtre principale

# Sélection du répertoire de travail
path = filedialog.askdirectory()

# Affichage du path choisi
print("Dossier sélectionné :", path)

Dossier sélectionné : /Users/timotheedangleterre/Desktop/projet_data_management/Datas


In [3]:
#Affectation du répertoire de travail au chemin sélectionné
os.chdir(path)

In [4]:
#Importation des fichiers text contenant les valeurs foncières entre 2018 et 2023
files = [os.path.join(path, file) for file in os.listdir(path)]

#Concaténation des fichiers pour récupérer la totalité des observations dans un dataframe
df_immo = pd.concat((pd.read_csv(file, sep = "|", header=0,low_memory=False) for file in files), sort=False)

In [5]:
# Dimensions of the data :
df_immo.shape

(19765458, 43)

In [6]:
# Printing the first rows
df_immo.head()

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


# Nettoyage des données
## 1) Traitement des données manquantes

Il y'a beaucoup de données manquantes sur ces 5 premières lignes. Par ailleurs, certaines colonnes semblent être entièrement vides. Pour le vérifier, il faut déterminer le nombre de données manquantes par colonnes :

In [8]:
# Identification des données manquantes pour chaque colonne
df_immo.isna().sum()

Identifiant de document       19765458
Reference document            19765458
1 Articles CGI                19765458
2 Articles CGI                19765458
3 Articles CGI                19765458
4 Articles CGI                19765458
5 Articles CGI                19765458
No disposition                       0
Date mutation                        0
Nature mutation                      0
Valeur fonciere                 196141
No voie                        7516144
B/T/Q                         18878772
Type de voie                   7925641
Code voie                       165571
Voie                            166146
Code postal                     166388
Commune                              0
Code departement                     0
Code commune                         0
Prefixe de section            18841052
Section                            699
No plan                              0
No Volume                     19716828
1er lot                       13517806
Surface Carrez du 1er lot

Choix : conserver seulement les colonnes ayant moins de 50% de valeurs manquantes.

In [9]:
# Fonction permettant de supprimer les colonnes intégralement vide
# Input : le dataframe contenant les données importées

def supp_vid(df):
    
    # Récupération du nombre de lignes du dataframe
    nrows = df.shape[0]
    
    # Récupération des intitulés des colonnes
    cols = df.columns
    
    # Création d'une liste vierge pour récupérer les colonnes à conserver
    cols_to_keep = []
    
    # Boucle
    for j in range(len(cols)):
        
        # Compte du nombre de NA pour chaque colonne
        nb_na = df[cols[j]].isna().sum()
        
        # Si le nombre de NA est différent du nombre de lignes, on récupère la colonnes
        if nb_na <= 0.5*nrows:
            cols_to_keep.append(cols[j])
    
    # Récupération du dataframe sans les colonnes vides
    df_non_empty = df[cols_to_keep]
    
    return df_non_empty

In [10]:
# Récupération du dataframe contenant les données importées sans les colonnes au moins
# à moitié vides
df_non_empty = supp_vid(df_immo)
df_non_empty.head()

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,1,07/01/2020,Vente,"8000,00",NaN,NaN,B063,FORTUNAT,1250.0,CEYZERIAT,...,72,AK,216,0,NaN,NaN,NaN,NaN,T,1061.0
1,1,02/01/2020,Vente,"2175,00",NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,LAIZ,...,203,B,4,0,NaN,NaN,NaN,NaN,BT,85.0
2,1,02/01/2020,Vente,"2175,00",NaN,NaN,B006,BOIS DU CHAMP RION,1290.0,LAIZ,...,203,B,173,0,NaN,NaN,NaN,NaN,T,1115.0
3,1,02/01/2020,Vente,"2175,00",NaN,NaN,B025,EN COROBERT,1290.0,LAIZ,...,203,B,477,0,NaN,NaN,NaN,NaN,T,1940.0
4,1,02/01/2020,Vente,"2175,00",NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,LAIZ,...,203,C,68,0,NaN,NaN,NaN,NaN,T,1148.0


In [11]:
# Vérification de la nouvelle taille du dataframe
df_non_empty.shape

(19765458, 21)

## 2) Gestion des doublons

In [29]:
# Suppression des doublons
#df_non_empty.drop_duplicates(inplace=True)
#df_non_empty.head()

/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_3132/3328064227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_empty.drop_duplicates(inplace=True)


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,1,07/01/2020,Vente,"8000,00",NaN,NaN,B063,FORTUNAT,1250.0,CEYZERIAT,...,72,AK,216,0,NaN,NaN,NaN,NaN,T,1061.0
1,1,02/01/2020,Vente,"2175,00",NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,LAIZ,...,203,B,4,0,NaN,NaN,NaN,NaN,BT,85.0
2,1,02/01/2020,Vente,"2175,00",NaN,NaN,B006,BOIS DU CHAMP RION,1290.0,LAIZ,...,203,B,173,0,NaN,NaN,NaN,NaN,T,1115.0
3,1,02/01/2020,Vente,"2175,00",NaN,NaN,B025,EN COROBERT,1290.0,LAIZ,...,203,B,477,0,NaN,NaN,NaN,NaN,T,1940.0
4,1,02/01/2020,Vente,"2175,00",NaN,NaN,B124,TERRES DES CINQ SAULES,1290.0,LAIZ,...,203,C,68,0,NaN,NaN,NaN,NaN,T,1148.0


In [30]:
# Nombres de lignes après la suppression des doublons
df_non_empty.shape

(17532869, 21)

In [35]:
# Colonnes restantes
df_non_empty.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Section', 'No plan',
       'Nombre de lots', 'Code type local', 'Type local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Surface terrain'],
      dtype='object')

## 2) Gestion des types de données

Un autre retraitement que nous allons également effectuer consiste à vérifier le type de données et à transformer les colonnes qui ne seraient pas au bon format (ex : Dates pas au format Date, nombre en string, ...). Pour cela, on étudie la structure de données de ce dataframe:

In [40]:
# Structure de données du dataframe
df_non_empty.dtypes

No disposition                 int64
Date mutation                 object
Nature mutation               object
Valeur fonciere               object
No voie                      float64
Type de voie                  object
Code voie                     object
Voie                          object
Code postal                  float64
Commune                       object
Code departement              object
Code commune                   int64
Section                       object
No plan                        int64
Nombre de lots                 int64
Code type local              float64
Type local                    object
Surface reelle bati          float64
Nombre pieces principales    float64
Nature culture                object
Surface terrain              float64
dtype: object

La majorité des colonnes sont de type "object". Ce type est utilisé par numpy lorsque les colonnes d'un dataframe sont constituées de chaînes de caractères. Par conséquent, une partie des colonnes n'a pas le bon type, notamment :
- La date de mutation
- Les valeurs foncières
- Les code de département
- Le type de bien dont il s'agit (Type local)

Il faudra donc modifier le type de ces variables pour pouvoir les traiter dans la suite du projet, notamment pour la construction du modèle de Machine Learning.

En outre, il faut sélectionner les colonnes pertinentes pour le modèle. Nous souhaitons évaluer l'évolution des prix de l'immobilier sur 5 ans par région / département et en fonction des taux. La conservation de certaines colonnes semble donc évidente :
- Date mutation
- Valeur foncière
- Surface terrain
- ...

Au delà de ces colonnes, pour étudier la répartition géographiques de ces transactions, on conserve :
- Le code postal
- Le code département
- Le code commune
- La commune

In [12]:
def prepare_data_for_analysis(df):
    # Selecting relevant columns
    columns_to_keep = [
        'Date mutation', 'Valeur fonciere', 'Code postal','Commune','Code departement','Code commune','Type local',
        'Surface reelle bati', 'Nombre pieces principales', 'Nature mutation', 'Surface terrain'
    ]
    df = df[columns_to_keep]
    
    # Data Cleaning Steps
    
    # Convert 'Date mutation' to datetime format
    df['Date mutation'] = pd.to_datetime(df['Date mutation'], format='%d/%m/%Y', errors='coerce')
    
    # Convert 'Valeur fonciere' to numeric, handling French formatting for numbers
    df['Valeur fonciere'] = pd.to_numeric(df['Valeur fonciere'].str.replace(',', '.').str.replace(' ', ''), errors='coerce')
    
    # Fill missing numeric values with the median and categorical values with the mode
    numeric_cols = ['Surface reelle bati', 'Nombre pieces principales', 'Surface terrain']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].median(), inplace=True)
    
    categorical_cols = ['Type local', 'Nature mutation']
    for col in categorical_cols:
        df[col].fillna(df[col].mode()[0], inplace=True)
    
    # Ensure 'Code postal' is correctly formatted (5 digits)
    df['Code postal'] = df['Code postal'].apply(lambda x: f"{int(x):05d}" if pd.notna(x) else np.nan)
    
    # Remove rows with critical missing information or incorrect data format
    df.dropna(subset=['Date mutation', 'Valeur fonciere', 'Code postal'], inplace=True)

    return df

In [13]:
# Création du dataframe contenant les données nettoyées
cleaned_data = prepare_data_for_analysis(df_non_empty)
cleaned_data.head()

/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_1681/894791442.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date mutation'] = pd.to_datetime(df['Date mutation'], format='%d/%m/%Y', errors='coerce')
/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_1681/894791442.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Valeur fonciere'] = pd.to_numeric(df['Valeur fonciere'].str.replace(',', '.').str.replace(' ', ''), errors='coerce')
/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r000

,Date mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Type local,Surface reelle bati,Nombre pieces principales,Nature mutation,Surface terrain
0,2020-01-07,8000.0,01250,CEYZERIAT,01,72,Dépendance,46.0,2.0,Vente,1061.0
1,2020-01-02,2175.0,01290,LAIZ,01,203,Dépendance,46.0,2.0,Vente,85.0
2,2020-01-02,2175.0,01290,LAIZ,01,203,Dépendance,46.0,2.0,Vente,1115.0
3,2020-01-02,2175.0,01290,LAIZ,01,203,Dépendance,46.0,2.0,Vente,1940.0
4,2020-01-02,2175.0,01290,LAIZ,01,203,Dépendance,46.0,2.0,Vente,1148.0


On vérifie la structure des données après ce nouveau retraitement :

In [14]:
cleaned_data.dtypes

Date mutation                datetime64[ns]
Valeur fonciere                     float64
Code postal                          object
Commune                              object
Code departement                     object
Code commune                          int64
Type local                           object
Surface reelle bati                 float64
Nombre pieces principales           float64
Nature mutation                      object
Surface terrain                     float64
dtype: object

In [18]:
cleaned_data.drop_duplicates(inplace=True)

/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_1681/3557917476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.drop_duplicates(inplace=True)


In [19]:
cleaned_data.shape

(16925381, 11)

On vérifie également les données départementales obtenues :

In [39]:
cleaned_data['Code departement'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31', '32',
       '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
       '55', '56', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '69', '70', '71', '72', '73', '74', '76', '77', '78', '79', '80',
       '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91',
       '92', '93', '94', '95', '971', '972', '973', '974', '75'],
      dtype=object)

On observe que le numéro de département est traité comme une chaîne de caractère. Cela s'explique par la présence de deux codes alphanumériques :
- "2A" : correspond à la Corse du Sud
- "2B" : correspond à la Haute Corse

Par ailleurs, il manque certains numéros de département :
- 57
- 67
- 68

Ces départements correspondent à la Moselle, au Bas-Rhin ainsi qu'au Haut-Rhin. Ils n'apparaissent pas car les données ne renseignent pas les transactions survenues sur ces 3 départements (voir data.gouv pour les raisons). 

Nous allons désormais chercher à re-traiter les types de logements présents dans la base (le nombre de lignes parait très élevé).

### Retraitement des types de logements 

On s'interroge sur le type de données immobilières présentes dans cette base, notamment sur la typologie des locaux (résidentiels ? commerciaux ? Maison ? Appartement ?, ...). Pour cela, on vérifie le type de logement et le nombre de transactions associées à chacun de ces types :

In [16]:
# On vérifie le type de logement présents dans la base :
cleaned_data['Type local'].unique()

array(['Dépendance', 'Maison', 'Appartement',
       'Local industriel. commercial ou assimilé'], dtype=object)

In [20]:
# On vérifie le nombre de lignes associés à chacun de ces types
cleaned_data.groupby('Type local').agg({'Date mutation':'count'})

,Date mutation
Type local,
Appartement,2757573
Dépendance,10026920
Local industriel. commercial ou assimilé,588971
Maison,3551917


Deux constants s'imposent :
- La base contient principalement des ventes de biens résidentiels. Il y a ainsi moins d'un million de lignes associées à des bureaux.
- L'immense majorité des lignes correspondent à des "dépendances". Dans le vocabulaire foncier, ce terme correspond à des constructions accessoires à la partie principale mais ne faisant pas partiême e du même groupement topographique. Ce terme regroupe notamment les piscines, garages, les jardins d'hiver, ... La distinction est opérée pour des raisons fiscales (calcul de la taxe foncière).

Puisque l'on s'intéresse à l'évolution du marché immobilier, on fait le choix de se concetrer exclusivement sur les Appartements et maisons, le volume de biens commerciaux étant ici peu importants (impact des taux a été très fort sur ce segment mais cela se réflète pôt àl uttravers d'autres indicateurs, notamment le taux de vances des immeubles). Nous retirons par ailleurs les lignes "Dépendance" qui ne correspondent pas réellement à des biens immobiliers.

In [21]:
# Base contenant uniquement les données immobilières résidentielle
df_immo_res = cleaned_data[(cleaned_data['Type local'] != "Dépendance") & (cleaned_data['Type local'] != "Local industriel. commercial ou assimilé")]

In [22]:
df_immo_res.head()

,Date mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Type local,Surface reelle bati,Nombre pieces principales,Nature mutation,Surface terrain
11,2020-01-09,72000.0,01270,COLIGNY,01,108,Maison,35.0,2.0,Vente,381.0
13,2020-01-06,180300.0,01000,BOURG-EN-BRESSE,01,53,Maison,75.0,4.0,Vente,525.0
14,2020-01-06,54800.0,01000,BOURG-EN-BRESSE,01,53,Appartement,32.0,1.0,Vente,612.0
16,2020-01-03,350750.0,01000,SAINT-DENIS-LES-BOURG,01,344,Maison,201.0,7.0,Vente,1497.0
18,2020-01-03,350750.0,01000,SAINT-DENIS-LES-BOURG,01,344,Maison,201.0,7.0,Vente,1267.0


In [23]:
# On a désormais plus que 6.3 millions de lignes. On vérifie la valeur foncière totale de la base obtenue :
montant = df_immo_res['Valeur fonciere'].sum()
montant

4984628667028.116

In [24]:
montant / df_immo_res.shape[0]

790020.8522444946

Suite à ce retraitement, la valeur totale des biens immobiliers vendus au cours des 5 dernières années est d'environ 4 979 Mds, soit un prix moyen par transaction de 789 000 €. Néanmoins, on constate que plusieurs lignes semblent associées aux mêmes opérations. Il y'a ainsi 2 opérations de 350 750 € survenues le premier mars 2020 à Saint-Denis les bourg. On peut supposer qu'il s'agit en réalité d'une seule transaction qui a été séparée sur deux lignes distinctes pour des raisons fiscales (à voir).

On propose donc de retraiter ces quasi-doublons en aggrégeant par la surface du terrain les lignes qui correspondent partiellement. Autrement, cela pourrait se traduire par des valeurs rapportées à la surface faussées.

Malgré ce retraitement, la valeur foncière totale reste de près de 4 979 milliards (soit un prix moyen par transaction de 789 000 euros. Ce montant est déjà plus raisonnable que ce que l'on observait précédemment mais reste très élevé au regard des prix des logements pratiqués sur l'ensemble du territoire. Par ailleurs, on observe dans la base que plusieurs lignes semblent associêmeées aux ms opératiêons (ex : opération du 3 janvier 2020 pour 350 750 euros, ...). On se propose donc de retraiter ces lignes en conservant la valeur foncière et en sommant les surfaces terrains afin d'avoir la surface totale associée à la vente. Autrement, des calculs de prix / surface seraient faussés.

In [25]:
# liste contenant toutes les colonnes sauf surface terrain par laquelle on va agréger les données
group_cols = df_immo_res.columns.tolist()
group_cols.remove('Surface terrain')
group_cols.remove('Surface reelle bati')
group_cols.remove('Nombre pieces principales')

# Création d'un dataframe regroupant les données par surface terrain.
df_immo_res_2 = df_immo_res.groupby(group_cols,sort=False, as_index=False).agg({
    'Surface terrain':'sum',
    'Surface reelle bati':'sum',
    'Nombre pieces principales':'sum'})

In [26]:
df_immo_res_2

,Date mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Type local,Nature mutation,Surface terrain,Surface reelle bati,Nombre pieces principales
0,2020-01-09,72000.0,01270,COLIGNY,01,108,Maison,Vente,381.0,35.0,2.0
1,2020-01-06,180300.0,01000,BOURG-EN-BRESSE,01,53,Maison,Vente,525.0,75.0,4.0
2,2020-01-06,54800.0,01000,BOURG-EN-BRESSE,01,53,Appartement,Vente,612.0,32.0,1.0
3,2020-01-03,350750.0,01000,SAINT-DENIS-LES-BOURG,01,344,Maison,Vente,2764.0,402.0,14.0
4,2020-01-10,53650.0,01270,COLIGNY,01,108,Appartement,Vente,390.0,95.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
5203575,2019-12-05,17521000.0,75004,PARIS 04,75,104,Appartement,Vente,5640.0,924.0,39.0
5203576,2019-10-10,610000.0,75004,PARIS 04,75,104,Appartement,Adjudication,612.0,44.0,2.0
5203577,2019-12-30,1400000.0,75002,PARIS 02,75,102,Appartement,Vente,612.0,97.0,3.0
5203578,2019-12-17,620000.0,75004,PARIS 04,75,104,Appartement,Adjudication,612.0,45.0,2.0


In [27]:
# Taille du dataframe obtenu
np.shape(df_immo_res_2)

(5203580, 11)

In [29]:
# Prix rapporté à la surface median
np.median(df_immo_res_2['Valeur fonciere']/df_immo_res_2['Surface reelle bati'])

2246.560470779221

Il reste désormais près de 5,2 millions de lignes dans le dataframe, soit une moyenne d'environ 1,04 millions de ventes par an. On vérifie la répartition entre appartements et maisons :

In [30]:
# Vérification de la segmentation Appartement / maison
df_immo_res_2.groupby('Type local').agg({'Date mutation':'count'})

,Date mutation
Type local,
Appartement,2340525
Maison,2863055


Les ventes entre maisons et appartements sont relativement équilibrées, les maisons représentent un volume des ventes légèrement plus important. 

On peut également s'intéresser à la nature de la mutation:

In [32]:
df_immo_res_2.groupby('Nature mutation').agg({'Date mutation':'count'})

,Date mutation
Nature mutation,
Adjudication,16045
Echange,7718
Expropriation,158
Vente,5038379
Vente en l'état futur d'achèvement,139886
Vente terrain à bâtir,1394


On choisit de conserver uniquement les ventes et ventes en l'état futur d'achèvement qui reflètent l'essentiel du marché immobilier français. Les cas d'expropriation notamment pourrait faire office de valeurs exêmes qtrui viendraient altérer les données

In [34]:
# On conserve seulement les ventes et les ventes en l'état futur d'achèvement
df_immo_res_2 = df_immo_res_2[(df_immo_res_2['Nature mutation']=="Vente")|(df_immo_res_2['Nature mutation']=="Vente en l'état futur d'achèvement")]
df_immo_res_2

,Date mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Type local,Nature mutation,Surface terrain,Surface reelle bati,Nombre pieces principales
0,2020-01-09,72000.0,01270,COLIGNY,01,108,Maison,Vente,381.0,35.0,2.0
1,2020-01-06,180300.0,01000,BOURG-EN-BRESSE,01,53,Maison,Vente,525.0,75.0,4.0
2,2020-01-06,54800.0,01000,BOURG-EN-BRESSE,01,53,Appartement,Vente,612.0,32.0,1.0
3,2020-01-03,350750.0,01000,SAINT-DENIS-LES-BOURG,01,344,Maison,Vente,2764.0,402.0,14.0
4,2020-01-10,53650.0,01270,COLIGNY,01,108,Appartement,Vente,390.0,95.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
5203573,2019-12-31,27200000.0,75004,PARIS 04,75,104,Appartement,Vente,1848.0,549.0,24.0
5203574,2019-12-30,680000.0,75002,PARIS 02,75,102,Appartement,Vente,612.0,72.0,3.0
5203575,2019-12-05,17521000.0,75004,PARIS 04,75,104,Appartement,Vente,5640.0,924.0,39.0
5203577,2019-12-30,1400000.0,75002,PARIS 02,75,102,Appartement,Vente,612.0,97.0,3.0


On observe qu'il existe une variété de mutations possibles. Pour homogénéiser les données, on fait le choix de ne conserver que les ventes et vente l'état

In [35]:
np.sum(df_immo_res['Surface terrain'])

5155523931.0

In [37]:
# Vérification de la nouvelle somme des valeurs foncières
montant = np.sum(df_immo_res_2['Valeur fonciere'])
montant

1411834129020.6604

In [38]:
# Calcul du prix moyen par transaction
prix_moy = montant / df_immo_res_2.shape[0]
prix_moy

272646.17183953704

On se retrouve avec un prix moyen d'environ 272 646 € par transaction. Ce chiffre semble déjà plus réaliste par rapport aux valeurs trouvées précédemment.

In [39]:
#Voir retraitements restants
#Parmi les colonnes à ajouter : mois / années
#Colonne pour les départements
#Colonne pour les régions
#Colonne pour permettre d'identifier les transactions
#Récupérer les taux d'intérêt pour chaque mois

#Nettoyer les transactions identiques (même transactions mais surfaces diff, ... donc pas détecté par drop duplicates)
#Vérifier que les codes postaux sont bien en phase avec les communes / dep, ... 

#Impact des taux d'intérêt: evolution du nombre de transactions dans le temps ?
#Regarder la valeur foncière quand les taux étaient très bas / depuis qu'ils sont hauts (= récupérer la valeur des taux par mois)

#Impact par région / département
#Nombre de transactions par région / département
#Impact selon le type de logement

#Etudier l'évolution de la valeur selon le type de logement

#Prix/surfance (équivalent prix par m2)

In [41]:
def revised_data_wrangling(df):
    # Assuming 'df' is a DataFrame with the selected columns after cleaning
    
    # Normalize 'Valeur fonciere' to range [0, 1]
    # This keeps all values positive and maintains interpretability
    min_val = df['Valeur fonciere'].min()
    max_val = df['Valeur fonciere'].max()
    df['Valeur fonciere'] = (df['Valeur fonciere'] - min_val) / (max_val - min_val)
    
    # Handle outliers in 'Surface reelle bati'
    # Instead of removing outliers, cap them at a certain percentile
    upper_limit = df['Surface reelle bati'].quantile(0.95)
    lower_limit = df['Surface reelle bati'].quantile(0.05)
    df['Surface reelle bati'] = np.clip(df['Surface reelle bati'], lower_limit, upper_limit)
    
    # Similarly, normalize 'Surface terrain' if needed
    min_surface = df['Surface terrain'].min()
    max_surface = df['Surface terrain'].max()
    df['Surface terrain'] = (df['Surface terrain'] - min_surface) / (max_surface - min_surface)
    
    # Ensure 'Code postal' is of string type and formatted correctly
    df['Code postal'] = df['Code postal'].astype(str).str.zfill(5)
    
    # Convert 'Type local' to categorical codes if machine learning models will be used
    df['Type local'] = df['Type local'].astype('category').cat.codes
    
    # Extract 'Year' and 'Month' from 'Date mutation' for temporal analysis
    df['Year'] = df['Date mutation'].dt.year
    df['Month'] = df['Date mutation'].dt.month

    return df

In [43]:
# Example usage, assuming 'cleaned_data' is the DataFrame after the cleaning process
wrangled_data = revised_data_wrangling(df_immo_res_2)
wrangled_data.head()

/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_1681/1964128194.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Valeur fonciere'] = (df['Valeur fonciere'] - min_val) / (max_val - min_val)
/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_1681/1964128194.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Surface reelle bati'] = np.clip(df['Surface reelle bati'], lower_limit, upper_limit)
/var/folders/zs/7r7lz82s5rbfdp6bwvpbhd3r0000gn/T/ipykernel_1681/1964128194.py:19: Settin

,Date mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Type local,Nature mutation,Surface terrain,Surface reelle bati,Nombre pieces principales,Year,Month
0,2020-01-09,0.000035,01270,COLIGNY,01,108,1,Vente,0.000006,35.0,2.0,2020,1
1,2020-01-06,0.000086,01000,BOURG-EN-BRESSE,01,53,1,Vente,0.000008,75.0,4.0,2020,1
2,2020-01-06,0.000026,01000,BOURG-EN-BRESSE,01,53,0,Vente,0.000009,32.0,1.0,2020,1
3,2020-01-03,0.000168,01000,SAINT-DENIS-LES-BOURG,01,344,1,Vente,0.000041,252.0,14.0,2020,1
4,2020-01-10,0.000026,01270,COLIGNY,01,108,0,Vente,0.000006,95.0,4.0,2020,1


In [ ]:
def find_outliers(x):
    q1 = x.quantile(.25)
    q3 = x.quantile(.75)
    iqr = q3 - q1
    floor = q1 - 1.5*iqr
    ceiling = q3 + 1.5*iqr
    outlier_indices = list(x.index[(x < floor) | (x > ceiling)])
    outlier_values = list(x[outlier_indices])
    return outlier_indices, outlier_values